In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from gensim import corpora
from gensim import models
import MeCab
from gensim.models import word2vec
from gensim.models import TfidfModel
from operator import itemgetter

/Users/mmiyazaki/dataiku/Design/DATA_DIR/code-envs/python/japan-nlp/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# Read recipe inputs
clean_trip_advisor = dataiku.Dataset("clean_trip_advisor")
df = clean_trip_advisor.get_dataframe()

model_folder_path = dataiku.Folder("m9JZdV7b").get_path()
model_path = model_folder_path + "/word2vec_ramen_model.model"
ramen_model = word2vec.Word2Vec.load(model_path)

wakati_folder = dataiku.Folder("0kM5kXKs").get_path()
tagger_path = '-Owakati -d ' + wakati_folder

In [4]:
df.head()

,en,jp,review
0,"We arrived for the opening time of 12h00, wait...",私たちは 12 時の開店時間に到着し、待ち行列で 25 分待ってから入りました。最終的に入る...,私たちは時の開店時間に到着し待ち行列で分待ってから入りました。最終的に入るまでにさらに分以上...
1,Real dissappointment total tourist trap. Food ...,本当の失望の総観光客の罠。食べ物は本当に塩辛く、味がしませんでした。他の 5 つ星のレビュー...,本当の失望の総観光客の罠。食べ物は本当に塩辛く味がしませんでした。他のつ星のレビューを購入し...
2,"The best ramen I’ve ever had. Tender meat, fre...",私が今まで食べた中で最高のラーメン。柔らかい肉、新鮮な食材、素晴らしいスープ、まともなバリエ...,私が今まで食べた中で最高のラーメン。柔らかい肉新鮮な食材素晴らしいスープまともなバリエーショ...
3,"Really excellent food, and fab service. Also r...",本当に素晴らしい料理と素晴らしいサービス。また、他のパリのレストランと比較して比較的安価です...,本当に素晴らしい料理と素晴らしいサービス。また他のパリのレストランと比較して比較的安価です。...
4,Definitely ramen worth putting on a bucket lis...,バケツリストに載せる価値のあるラーメン。すべての素材の組み合わせによる味と香りの完璧さは、神...,バケツリストに載せる価値のあるラーメン。すべての素材の組み合わせによる味と香りの完璧さは神体...


In [6]:
tagger = MeCab.Tagger(tagger_path)#タグはMeCab.Tagger（neologd辞書）を使用
tagger.parse('')

'\n'

In [7]:
def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"]:#分かち書きで取得する品詞を指定
            yield node.surface.lower()
        node = node.next
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [8]:
#コーパス作成
wakati_ramen_text = []
for i in df['review']:
    txt = tokenize(i, 2, 10000, True)
    wakati_ramen_text.append(txt)

In [9]:
wakati_ramen_text

[['私たち',
  '開店',
  '時間',
  '到着',
  '待ち行列',
  '最終的',
  '以上',
  '待ち時間',
  '食べ物',
  '例外',
  'それ',
  'フランス',
  '食材',
  'はず',
  '放し飼い',
  'これ',
  '食べ物',
  '交差',
  'こと',
  '唯一',
  '肯定',
  '側面',
  '本物',
  '経験',
  '雰囲気',
  'さら',
  '表示'],
 ['本当',
  '失望',
  '観光客',
  '食べ物',
  '塩辛く',
  'レビュー',
  '購入',
  'レストラン',
  '観光客',
  '生活',
  'レビュー',
  'ただ',
  'これ',
  '悲しい'],
 ['最高',
  'ラーメン',
  '柔らかい',
  '新鮮',
  '食材',
  '素晴らしい',
  'スープ',
  'まとも',
  'バリエーション',
  'スープ',
  'もの',
  'どちら',
  '美味しかっ',
  '市場通り',
  'よう',
  '窮屈',
  '不快'],
 ['素晴らしい',
  '料理',
  '素晴らしい',
  'サービス',
  'パリのレストラン',
  '比較',
  '安価',
  'パリ',
  'とき',
  '私のお気に入り',
  'レストラン',
  '混雑',
  '閉店',
  '直前',
  'おすすめ'],
 ['バケツ',
  'リスト',
  '価値',
  'ラーメン',
  'すべて',
  '素材',
  '組み合わせ',
  '香り',
  '完璧',
  '神体',
  '参加',
  'よう',
  'インテリア',
  '私たち',
  '時間',
  '現実',
  '認識',
  '明らか',
  '追加',
  '側面',
  '提供'],
 ['すべて',
  'レビュアー',
  'こと',
  'ため',
  '約分',
  'これ',
  '素晴らしい',
  'ラーメン',
  '最適',
  '場所',
  '風味',
  '豊か',
  'ストック',
  'レストラン',
  '露店',
  'よう',
  '装飾',
  'レストラン',

In [0]:
# max_vocab = 30000 #40000にしても結果は同じだった
# vocab = list(word2vec_ramen_model.wv.vocab.keys())[:max_vocab]
vectors = [word2vec_ramen_model.wv[word] for word in wakati_ramen_text]

vocab_df = pd.DataFrame(vectors)
vocab_df['words'] = vocab

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

trip_advisor_vectorized_df = clean_trip_advisor_df # For this sample code, simply copy input to output


# Write recipe outputs
trip_advisor_vectorized = dataiku.Dataset("trip_advisor_vectorized")
trip_advisor_vectorized.write_with_schema(trip_advisor_vectorized_df)